In [2]:
#For this problem, you aren't writing any code.
#Instead, please just change the last argument 
#in f() to maximize the output.

from math import *

def f(mu, sigma2, x):
    return 1/sqrt(2.*pi*sigma2) * exp(-.5*(x-mu)**2 / sigma2)

print (f(10.,4.,10.)) #Change the 8. to something else!


0.19947114020071635


In [4]:
# Write a program to update your mean and variance
# when given the mean and variance of your belief
# and the mean and variance of your measurement.
# This program will update the parameters of your
# belief function.

def update(mean1, var1, mean2, var2):
    new_mean = 1/(var1+var2)*(var2*mean1 + var1*mean2)
    new_var = 1/(1/var1+1/var2)
    return [new_mean, new_var]

print (update(10.,8.,13., 2.))

[12.4, 1.6]


In [5]:
# Write a program that will predict your new mean
# and variance given the mean and variance of your 
# prior belief and the mean and variance of your 
# motion. 

def update(mean1, var1, mean2, var2):
    new_mean = (var2 * mean1 + var1 * mean2) / (var1 + var2)
    new_var = 1/(1/var1 + 1/var2)
    return [new_mean, new_var]

def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2        
    new_var = var1 + var2
    return [new_mean, new_var]

print (predict(10., 4., 12., 4.))

[22.0, 8.0]


In [7]:
# Write a program that will iteratively update and
# predict based on the location measurements 
# and inferred motions shown below. 

def update(mean1, var1, mean2, var2):
    new_mean = float(var2 * mean1 + var1 * mean2) / (var1 + var2)
    new_var = 1./(1./var1 + 1./var2)
    return [new_mean, new_var]

def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4.
motion_sig = 2.
mu = 0.
sig = 10000.

#Please print out ONLY the final values of the mean
#and the variance in a list [mu, sig]. 

# Insert code here
for k in range(len(measurements)):
    [mu, sig] = update(mu, sig, measurements[k], measurement_sig)
    #print("update", [mu, sig])
    [mu, sig] = predict(mu, sig, motion[k], motion_sig)
    #print("prediction", [mu, sig])


print ([mu, sig])


[10.999906177177365, 4.005861580844194]
